In [ ]:
"""
    generate_log_spaced_times(L, z, num_points, regions; ...)

Generates logarithmically spaced measurement times, snapping them to interval-based
grids defined by different regions.

# Arguments
- `regions`: A vector of tuples `(t_threshold, interval)`. Each tuple defines that
  for times `t < t_threshold`, the measurement times will be multiples of `interval`.
  The regions should be sorted by `t_threshold`. The last threshold should be `Inf`.

# Example
`regions = [(100, 1), (1000, 10), (Inf, 100)]` means:
- `t < 100`: interval is 1 (measure every step)
- `100 <= t < 1000`: interval is 10
- `t >= 1000`: interval is 100
"""
function generate_log_spaced_times(
    L::Int, 
    z::Float64, 
    num_points::Int,
    regions::Vector; 
    max_scaled_time::Float64=0.5, 
    t_min::Int=1
)
    t_max = max_scaled_time * (L^z)
    if t_max < t_min
        return [t_min]
    end

    # 1. Generate the "ideal" logarithmically spaced points
    log_steps = range(log10(t_min), stop=log10(t_max), length=num_points)
    tmea_float = 10 .^ log_steps

    snapped_times = Int[]
    for t in tmea_float
        # 2. For each ideal point, find which region it falls into
        interval = 1
        for (threshold, intrvl) in regions
            if t < threshold
                interval = intrvl
                break
            end
        end
        
        # 3. Snap the ideal point to the nearest multiple of the determined interval
        snapped_t = round(Int, t / interval) * interval
        
        # Ensure the snapped time is at least t_min
        push!(snapped_times, max(t_min, snapped_t))
    end

    # 4. Return the unique, sorted list of valid measurement times
    return sort(unique(snapped_times))
end